In [1]:
import pandas as pd

from interfacedb import setup_lbnl

## Initiate a database object

In [2]:
db, dbname = setup_lbnl()
# db is an instance of the database object
# dbname is a database (like a table), a server can host multiple databases 

In [3]:
dbname

'ucmerced'

In [4]:
# get the measurement lists in this database
db.get_measurements_list(dbname)

['MOER', 'MOER_for']

## Demonstration: historical data

In [5]:
from gridCarbon import WattTime

## Initiate an object for the downloader
LOC = [37.87, -122.27]  # Berkeley, CA
wattTime = WattTime(LOC)

## Download the historical MOER
# This function needs Analyst account
# Return dataframe
START_UTC = '2021-2-01 00:00:00' 
FINAL_UTC = '2021-2-05 12:00:00'
moer_historical = wattTime.MOER_historical(START_UTC, FINAL_UTC, mode='Test')

In [15]:
moer_historical.rename(columns={'MOER':'MOER_test'}, inplace=True)
moer_historical  # the data to be written to the database

In [17]:
db.write_data(moer_historical, dbname)  # moer_historical needs to be pd dataframe

1

In [18]:
## print the existing data point in the database
db.get_measurements_list(dbname)

['MOER', 'MOER_for', 'MOER_test']

In [21]:
## read the data
field = 'MOER_test'
start_time_str = '2020-1-1 00:00:00'
final_time = pd.Timestamp.now(tz='UTC')
final_time_str = pd.to_datetime(final_time.strftime('%Y-%m-%d %H:%M:00'))

# the start and final time must be string and suggested to be UTC to avoid mistakes
moer_data = db.get_data(field, dbname, start_time_str, final_time_str)

In [22]:
moer_data

,MOER_test
2021-02-01 00:00:00+00:00,983.0
2021-02-01 00:05:00+00:00,984.0
2021-02-01 00:10:00+00:00,949.0
2021-02-01 00:15:00+00:00,940.0
2021-02-01 00:20:00+00:00,953.0
...,...
2021-02-05 11:40:00+00:00,977.0
2021-02-05 11:45:00+00:00,976.0
2021-02-05 11:50:00+00:00,975.0
2021-02-05 11:55:00+00:00,969.0


## Demonstration: prediction data

In [23]:
# As forecast is updated at every time step, however we do not want the forecast to be over-written
# Therefore, we propose a seperate database API to write and save forecasts data
# You may not necessary have to use it
db_for, dbname = setup_lbnl(style='forecasts')

In [24]:
# collect the forecast data
moer_forecast = wattTime.MOER_forecast(mode='Test')
moer_forecast.rename(columns={field: field+"_for" for field in moer_forecast.columns}, inplace=True)

Using the Test mode, the location you indicate is not available, only CAISO_NORTH is available


In [25]:
moer_forecast

,MOER_for
2021-02-26 21:50:00+00:00,49.117397
2021-02-26 21:55:00+00:00,50.325799
2021-02-26 22:00:00+00:00,50.745274
2021-02-26 22:05:00+00:00,53.476336
2021-02-26 22:10:00+00:00,50.617549
...,...
2021-02-27 21:25:00+00:00,74.133299
2021-02-27 21:30:00+00:00,74.113806
2021-02-27 21:35:00+00:00,71.796273
2021-02-27 21:40:00+00:00,72.708373


In [26]:
db_for.write_data(moer_forecast, dbname, time = now_str)

In [ ]:
# You only need one timestamp (the time step you saved the forecast) to retrieve the forecast
# Remember for historical data, you need two timestamps (start time and final time)
db_for.get_data('MOER_for', dbname, now_str)

In [14]:
## We can also delete a point, but requires an admin privilege to delete data
db.drop_measurement('MOER', dbname)

InfluxDBClientError: 403: {"error":"error authorizing query: cerc_user not authorized to execute statement 'DROP MEASUREMENT MOER', requires admin privilege"}